In [ ]:
#ライブラリのインポート
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pyplot as p目的変数
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
import warnings
from sklearn.ensemble import RandomForestClassifier
warnings.fi目的変数erwarnings('ignore')#warningを表示させない設定
 
from sklearn import preprocessing
 
from sklearn.ensemble import RandomForestRegressor
 
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
 
import matplotlib.pyplot as p目的変数
p目的変数.rcParams['font.size'] = 18
%matplotlib inline
import shap
from operator import 目的変数
import optuna as opt
from functools import partial


pd.set_option('display.max_columns', None)

In [ ]:
#データの読み込み
data=pd.read_csv("data.csv")

In [ ]:
#data = pd.read_csv("fin.csv")#データの書き出し用

In [ ]:
#データの中身確認
data

In [ ]:
#データの複製
dat =data.copy()

In [ ]:
#データの中身確認
dat.shape

In [ ]:
#Na値の除去
dat=dat.dropna()

In [ ]:
#Null値の除去
dat.isnull().sum()

In [ ]:
#目的変数の確認
dat[["目的変数"]]

In [ ]:
#ヒストグラムの出力
p目的変数.axvline(x=dat["目的変数"].mean(),color="red")
dat["目的変数"].plot.hist(grid=True)

In [ ]:
#ヒストグラムの出力
sns.distplot(dat.目的変数)

In [ ]:
#目的変数の設定
target = "目的変数"

In [ ]:
#カテゴリ分けで変数に格納
変数１=['カラム１','カラム２','カラム３']
変数２=['カラム４','カラム５']
変数３=['カラム６']
features=変数1+ 変数2 + 変数3

In [ ]:
#カラム名ごとに説明変数の設定
x = dat.loc[:,features]
#目的変数の設定
y = dat.iloc[:, 0].values

In [ ]:
#ランダムフォレストの設定
n_estimators = 5
min_samples_leaf = 500
min_samples_split = 2
max_depth = 40
ccp_alpha = 0.3
bootstrap = True

scores = []

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)


params={"n_estimators": n_estimators,
            "min_samples_leaf":min_samples_leaf, 
            "min_samples_split" : min_samples_split,
            "max_depth" : max_depth, 
            "ccp_alpha" : ccp_alpha, 
            "bootstrap" : bootstrap
            }

rf = RandomForestRegressor(random_state=0, **params)
rf.fit(x_train, y_train)
prediction = rf.predict(x_test)

rmse = mean_squared_error(y_true = y_test, y_pred = prediction, squared = False)
scores.append(rmse)

In [ ]:
#変数がどれだけ重要かを値として表示
rf.feature_importances_

In [ ]:
prediction = rf.predict(x)

In [ ]:
scores

In [ ]:
dat["目的変数"].mean()

In [ ]:
for score in scores:
    print(score/dat["目的変数"].mean())

In [ ]:
def nrmse(y_true, y_pred):#nrmse ルートをした結果がかえる
    return np.sqrt(np.mean((y_true - y_pred) ** 2)) / (np.max(y_true) - np.min(y_true))#np.sqrt ルートをとる
    #(np.max(y_true) - np.min(y_true))正規化を行う

In [ ]:
rmse_metric = mean_squared_error(y_true = dat["目的変数"], y_pred = prediction, squared=False)
mape_metric = mean_absolute_percentage_error(y_true = dat["目的変数"], y_pred = prediction)
nrmse_metric = nrmse(dat["目的変数"],  prediction)
r2_metric = r2_score(y_true = dat["目的変数"], y_pred = prediction)

fig, ax = p目的変数.subplots(figsize = (25, 8))
_ = ax.plot(prediction, color = "blue", label = "predicted")
_ = ax.plot(dat["目的変数"], 'ro', label = "true")
_ = p目的変数.title(f"RMSE: {np.round(rmse_metric)}, NRMSE: {np.round(nrmse_metric, 3)}, MAPE: {np.round(mape_metric, 3)}, R2: {np.round(r2_metric,3)}")
_ = ax.legend()

In [ ]:
explainer = shap.TreeExplainer(rf)
# SHAP valuesを取得
shap_values_train = explainer.shap_values(x)
# create a dataframe of the shap values for the training set and the test set
df_shap_values = pd.DataFrame(shap_values_train, columns=features)#columns=説明数のカラムを設定

In [ ]:
#絶対値からの平均値をとり、グラフ化
np.abs(df_shap_values).mean().plot.barh()
p目的変数.tight_layout()
print(np.abs(df_shap_values).mean()/ np.abs(df_shap_values).mean().sum())#相対的な占有率を表示

In [ ]:
importance = pd.DataFrame(rf.feature_importances_)
importance.index = features
importance.plot.barh()
print(importance)

In [ ]:
#全てのカラムで相関関係の確認

corlist = []
for feature in features:
    print(feature)
    #print(np.corrcoef(df_shap_values[feature], data.reset_index().drop("index", axis=1)[feature])[1][0])
    cornum = np.corrcoef(df_shap_values[feature], dat.reset_index().drop("index", axis=1)[feature])[1][0]
    print(cornum)
    corlist.append( np.where(cornum >0, "red", "blue"))
    print("")

In [ ]:
pd.DataFrame(corlist)

In [ ]:
p目的変数.tight_layout()
p目的変数.show()
np.abs(df_shap_values).mean().plot.barh(color=pd.DataFrame(corlist)[0])

In [ ]:
def shap_feature_importance(shap_values, dat,feature,figsize = (20, 10)):
    
    feature_list = dat.columns
    
    if isinstance(shap_values, pd.DataFrame) == False:
        shap_v = pd.DataFrame(shap_values)
        shap_v.columns = feature_list
    else:
        shap_v = shap_values
    
        
    df_v = dat.copy().reset_index().drop('index',axis=1)

    
    # 元データとSHAPの相関
    corr_list = list()
    for i in feature_list:
        b = np.corrcoef(shap_v[i],df_v[i])[1][0]
        corr_list.append(b)
    corr_df = pd.concat([pd.Series(feature_list),pd.Series(corr_list)],axis=1).fillna(0)
    # データフレームの作成（相関とカラーサインの作成）
    corr_df.columns  = ['Variable','Corr']
    corr_df['Sign'] = np.where(corr_df['Corr']>0,'red','blue')
    
    # 可視化
    shap_abs = np.abs(shap_v)
    k=pd.DataFrame(shap_abs.mean()).reset_index()
    k.columns = ['Variable','SHAP_abs']
    k2 = k.merge(corr_df,left_on = 'Variable',right_on='Variable',how='inner')
    k2 = k2.sort_values(by='SHAP_abs',ascending = True)
    colorlist = k2['Sign']
    ax = k2.plot.barh(x='Variable',y='SHAP_abs',color = colorlist, figsize=figsize,legend=False)
    ax.set_xlabel("SHAP Value (Red = Positive Impact)")

In [ ]:
#SHAP値の可視化
shap_feature_importance(df_shap_values, dat[features], features)

In [ ]:
p目的変数.scatter(dat["Fukuoka"], dat["目的変数"])#散布図
p目的変数.xlabel("Fukuoka")
p目的変数.ylabel("目的変数")

In [ ]:
p目的変数.scatter(dat["User Profile__marriage_intention"], dat["目的変数"])
p目的変数.xlabel("User Profile__marriage_intention")
p目的変数.ylabel("目的変数")

In [ ]:
p目的変数.scatter(dat["User Preference__appearance_priority"], dat["目的変数"])
p目的変数.xlabel("User Preference__appearance_priority")
p目的変数.ylabel("目的変数")

In [ ]:
responses = pd.DataFrame(df_shap_values[features].abs().sum(axis = 0), columns = ["effect_share"])
response_percentages = responses / responses.sum()
response_percentages

spends_percentages = pd.DataFrame(dat[features].sum(axis = 0) / dat[features].sum(axis = 0).sum(), columns = ["spend_share"])
spends_percentages

spend_effect_share = pd.merge(response_percentages, spends_percentages, left_index = True, right_index = True)
spend_effect_share = spend_effect_share.reset_index().rename(columns = {"index": "features"})

In [ ]:
spend_effect_share

In [ ]:
#結果の可視化
sns.set()
sns.set_style('whitegrid')
sns.set_palette('Paired')

x = np.array(spend_effect_share["features"].unique())#x_position = np.arange(len(x))


y_control = np.array(spend_effect_share["spend_share"])
y_stress = np.array(spend_effect_share["effect_share"])

fig = p目的変数.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1)
ax.bar(x_position, y_control,width=0.4,  label='spend share')
ax.bar(x_position + 0.4,  y_stress, width=0.4, label='effect share')
ax.legend()
ax.set_xticks(x_position + 0.2)
ax.set_xticklabels(x)
p目的変数.show()


In [ ]:
#結果の可視化
def plot_shap_vs_spend(df_shap_values, x_input_interval_nontransformed, x_input_interval_transformed, features, user_profile, figsize=(25, 10)):
    for profile in user_profile:
    
        #index = features.index(feature)

        mean_spend = x_input_interval_nontransformed.loc[x_input_interval_nontransformed[profile] > 0, profile].mean()

        fig, ax = p目的変数.subplots(figsize=figsize)
        sns.regplot(x = x_input_interval_transformed[profile], y = df_shap_values[profile], label = profile,
                    scatter_kws={'alpha': 0.65}, line_kws={'color': 'C2', 'linewidth': 6},
                    lowess=True, ax=ax).set(title=f'{profile}: Spend vs Shapley')
        ax.axhline(0, linestyle = "--", color = "black", alpha = 0.5)
        ax.axvline(mean_spend, linestyle = "--", color = "red", alpha = 0.5, label=f"Average Spend: {int(mean_spend)}")
        ax.set_xlabel(f"{profile} spend")
        ax.set_ylabel(f'SHAP Value for {profile}')
        p目的変数.legend()

In [ ]:
plot_shap_vs_spend(df_shap_values, dat[features], dat[features], features, 説明変数)